<a href="https://colab.research.google.com/github/drtophop/CASAML-Thesis/blob/main/CAS_AML_Thesis_NguyenHop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CAS AML Thesis

## Introduction

### Scope

### Contents

### Sources

### Remarks

## Libraries and Modules

#### Numpy

#### Opendatasets

The opendatasets Python package, provides a convenient way to download and work with open datasets from various sources

In [ ]:
!pip install opendatasets

## Datasets

#### Local Folder

In [ ]:
path_main = os.getcwd()

### Object Recognition

### Object Segmentation

## Object Recoginition with Yolov5

In [ ]:
# Prepare the String for the detection dataset folder
path_yoloset = f'{path_main}/car-object-detection/data'

##### Clear locals

Before starting with the Yolov5, existing contents and existing folders from a potentional former execution of this Notebook will be cleared to show proper execution, by following expressions

In [ ]:
dirs_to_delete = ['car-object-detection', 'yolov5', 'test', 'tmp', 'train', 'valid', 'yolov5_train']

for dir_to_delete in dirs_to_delete:
    path_del = os.path.join(path_main, dir_to_delete)
    if os.path.exists(path_del):
        shutil.rmtree(path_del)

files_to_delete = ['data.yaml','yolov5m6.pt']

for file_to_delete in files_to_delete:
    file_del = os.path.join(path_main, file_to_delete)
    if os.path.exists(file_del):
        os.remove(file_del)

### Clone and Setup Yolo V5

In [ ]:
# Clone yolov5 from the corresponding Git Repository to Local
!git clone https://github.com/ultralytics/yolov5

In [ ]:
# Install the required Python packages specified in the requirements.txt file within the yolov5 directory
!pip install -qr yolov5/requirements.txt

### Initialization

In [ ]:
PROJECT_NAME = "yolov5_train"
BASE_MODEL = "yolov5m6.pt"
TRAIN_BATCH = 32
TRAIN_EPOCHS = 20
VAL_BATCH = 64


# Doublecheck if needed Folders are ok
print(f'does path_main exist: {os.path.exists(path_main)}')
print(f'does path_yoloset exist: {os.path.exists(path_yoloset)}')